# Reading the Model and adding reactions

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite
import cobra.test
import os
from os.path import join

In [7]:
model = read_sbml_model('iML1515.xml.gz')
model

Name,iML1515
Memory address,0x0211a320a460
Number of metabolites,1877
Number of reactions,2712
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


## SucD Model

SucD model is a E.coli model in which 1,4 - butanediol biosynthesis pathway starts from succinate. Activation of succinate to succinyl-CoA is a naturaly occuring reaction, already present in the iML1515 model, so the added reactions start from the reduction step catalyzed by CoA dependent succinate semialdehyde dehydrogenase. Alcohol dehydrogense (last enzyme of the pathway) is present in iML1515 model, but this particular reaction it can catalyze is not, so it was added as last step in biosynthesis pathway. 

In [4]:
model_sucd= model.copy()
model_sucd

Name,iML1515
Memory address,0x07f9b2d7b3a00
Number of metabolites,1877
Number of reactions,2712
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


In [5]:
sucd = Reaction('SUCD') #CoA-dependent succinate semialdehyde dehydrogenase
succinyl_semialdehyde = Metabolite(id='succinyl_semialdehyde_c', compartment='c')

sucd.add_metabolites({model_sucd.metabolites.h_c: 1,
                              model_sucd.metabolites.nadh_c: -1,
                              model_sucd.metabolites.succoa_c: -1,
                              succinyl_semialdehyde: 1, #new one
                              model_sucd.metabolites.coa_c: 1,
                              model_sucd.metabolites.nad_c: 1
                             })
print(sucd.build_reaction_string())
model_sucd.add_reactions([sucd])
model_sucd.reactions.SUCD

nadh_c + succoa_c --> coa_c + h_c + nad_c + succinyl_semialdehyde_c


Reaction identifier,SUCD
Name,
Memory address,0x07f9b316bd430
Stoichiometry,nadh_c + succoa_c --> coa_c + h_c + nad_c + succinyl_semialdehyde_c Nicotinamide adenine dinucleotide - reduced + Succinyl-CoA --> Coenzyme A + H+ + Nicotinamide adenine dinucleotide +
GPR,
Lower bound,0.0
Upper bound,1000.0


In [6]:
fhbd = Reaction('FHBD') #4-hydroxybutyrate dehydrogenase
four_hydroxybutyrate = Metabolite(id='four_hydroxybutyrate_c', compartment='c')

fhbd.add_metabolites({model_sucd.metabolites.h_c: 1,
                              model_sucd.metabolites.nadh_c: -1,
                              succinyl_semialdehyde: -1, 
                              four_hydroxybutyrate: 1, #new one
                              model_sucd.metabolites.nad_c: 1
                             })
print(fhbd.build_reaction_string())
model_sucd.add_reactions([fhbd])
model_sucd.reactions.FHBD

nadh_c + succinyl_semialdehyde_c --> four_hydroxybutyrate_c + h_c + nad_c


Reaction identifier,FHBD
Name,
Memory address,0x07f9b2d7b3ac0
Stoichiometry,nadh_c + succinyl_semialdehyde_c --> four_hydroxybutyrate_c + h_c + nad_c Nicotinamide adenine dinucleotide - reduced + --> + H+ + Nicotinamide adenine dinucleotide
GPR,
Lower bound,0.0
Upper bound,1000.0


In [7]:
hbd_coa_t = Reaction('HBCT') #4-hydroxybutyryl-CoA transferase
four_hbcoa = Metabolite(id='four_hbcoa_c', compartment='c')
hbd_coa_t.add_metabolites({model_sucd.metabolites.accoa_c: -1,
                              model_sucd.metabolites.ac_c: 1,
                              four_hbcoa: 1, #new one
                              four_hydroxybutyrate: -1
                             })
print(hbd_coa_t.build_reaction_string())
model_sucd.add_reactions([hbd_coa_t])
model_sucd.reactions.HBCT

accoa_c + four_hydroxybutyrate_c --> ac_c + four_hbcoa_c


Reaction identifier,HBCT
Name,
Memory address,0x07f9b316b3880
Stoichiometry,accoa_c + four_hydroxybutyrate_c --> ac_c + four_hbcoa_c Acetyl-CoA + --> Acetate +
GPR,
Lower bound,0.0
Upper bound,1000.0


In [8]:
hbd_coa_r = Reaction('HBCR') #4-hydroxybutyryl-CoA reductase
four_hbaldehyde = Metabolite(id='four_hbaldehyde_c', compartment='c')
hbd_coa_r.add_metabolites({model_sucd.metabolites.h_c: 1,
                              model_sucd.metabolites.nadh_c: -1,
                              four_hbaldehyde: 1, #new one
                              four_hbcoa: -1,
                              model_sucd.metabolites.nad_c: 1,
                              model_sucd.metabolites.coa_c: 1
                             })
print(hbd_coa_r.build_reaction_string())
model_sucd.add_reactions([hbd_coa_r])
model_sucd.reactions.HBCR

four_hbcoa_c + nadh_c --> coa_c + four_hbaldehyde_c + h_c + nad_c


Reaction identifier,HBCR
Name,
Memory address,0x07f9b316e2e20
Stoichiometry,four_hbcoa_c + nadh_c --> coa_c + four_hbaldehyde_c + h_c + nad_c + Nicotinamide adenine dinucleotide - reduced --> Coenzyme A + + H+ + Nicotinamide adenine dinucleotide
GPR,
Lower bound,0.0
Upper bound,1000.0


In [9]:
adh= Reaction('ADH') #alcohol dehydrogenase
bdo = Metabolite(id='bdo_c', compartment='c')
adh.add_metabolites({model_sucd.metabolites.h_c: 1,
                              model_sucd.metabolites.nadh_c: -1,
                              four_hbaldehyde: -1,
                              bdo: 1, #new one
                              model_sucd.metabolites.nad_c: 1,
                             })
print(adh.build_reaction_string())
model_sucd.add_reactions([adh])
model_sucd.reactions.ADH

four_hbaldehyde_c + nadh_c --> bdo_c + h_c + nad_c


Reaction identifier,ADH
Name,
Memory address,0x07f9b316e2ee0
Stoichiometry,four_hbaldehyde_c + nadh_c --> bdo_c + h_c + nad_c + Nicotinamide adenine dinucleotide - reduced --> + H+ + Nicotinamide adenine dinucleotide
GPR,
Lower bound,0.0
Upper bound,1000.0


In [10]:
bdo_exchange = Reaction('EX_bdo')
bdo_exchange.add_metabolites({model_sucd.metabolites.bdo_c: -1})
model_sucd.add_reaction(bdo_exchange)
model_sucd.add_boundary(model_sucd.metabolites.bdo_c, type='demand')

Reaction identifier,DM_bdo_c
Name,demand
Memory address,0x07f9b316e2b20
Stoichiometry,bdo_c --> -->
GPR,
Lower bound,0
Upper bound,1000.0


At last, Sucd model is saved as a .json file in order to later visualize the pathway using the online version of Escher. 

In [11]:
cobra.io.save_json_model(model_sucd, "sucd.json")

### Yield Calulation 

Here the yield of Sucd Model is calculated. The units of the yield correspont to (producted/glucose)*mmol. It is an uptake reaction that is also why the calculation of the uptake is negative. It means that the reaction goes from the right to the left.

In [12]:
with model_sucd:
    model_sucd.objective=model_sucd.reactions.ADH
    optimized = float(model_sucd.optimize().objective_value)
    print(optimized) #flux mmol/gDWh how fast i produce 

9.605894736842117


In [13]:
uptake = model_sucd.reactions.EX_glc__D_e.flux #uptake <--
print("Uptake:",uptake) #flux of glucose unit: mmol/gDWh

Uptake: -10.0


In [14]:
Models_yield = dict()
Models_yield['sucd']= optimized
yield_sucd =(-1)*Models_yield['sucd']/uptake

In [15]:
print("Yield for model SucD =",yield_sucd) # Yield for Model_SucD (Succinate) p-mmol/gl-mmol
#it is multiplied by -1 because the reaction is an uptake which means thats goes for the right to the left
#we divde the the flux rate of produced yield with the uptake in order to see how much substrate has been used during the reactions.

Yield for model SucD = 0.9605894736842118


## Akg Model

Akg model is a E.coli model in which 1,4 - butanediol biosynthesis pathway starts from alpha - ketoglutarate. All of the reactions in this pathway are not native to E.coli, with the exception of naturally occuring last enzyme (alcohol dehydrogenase). While the enzyme is present in the cells, for the model to work 4-hydroxybutyraldehyde (substrate) and 1,4 - butanediol (product) needed to be connected to this enzyme. 

In [8]:
model_akg=model.copy()
model_akg

Name,iML1515
Memory address,0x0211a320af10
Number of metabolites,1877
Number of reactions,2712
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


In [9]:
oxodc = Reaction('OXODC') #2-oxoglutarate decarboxylase
succinyl_semialdehyde = Metabolite(id='succinyl_semialdehyde_c', compartment='c')

oxodc.add_metabolites({model_akg.metabolites.akg_c: -1,
                              succinyl_semialdehyde: 1, #new one
                              model_akg.metabolites.co2_c: 1
                             })
print(oxodc.build_reaction_string())
model_akg.add_reactions([oxodc])
model_akg.reactions.OXODC

akg_c --> co2_c + succinyl_semialdehyde_c


Reaction identifier,OXODC
Name,
Memory address,0x0211b6336100
Stoichiometry,akg_c --> co2_c + succinyl_semialdehyde_c 2-Oxoglutarate --> CO2 CO2 +
GPR,
Lower bound,0.0
Upper bound,1000.0


In [10]:
fhbd = Reaction('FHBD') #4-hydroxybutyrate dehydrogenase
four_hydroxybutyrate = Metabolite(id='four_hydroxybutyrate_c', compartment='c')

fhbd.add_metabolites({model_akg.metabolites.h_c: 1,
                              model_akg.metabolites.nadh_c: -1,
                              succinyl_semialdehyde: -1, 
                              four_hydroxybutyrate: 1, #new one
                              model_akg.metabolites.nad_c: 1
                             })
print(fhbd.build_reaction_string())
model_akg.add_reactions([fhbd])
model_akg.reactions.FHBD

nadh_c + succinyl_semialdehyde_c --> four_hydroxybutyrate_c + h_c + nad_c


Reaction identifier,FHBD
Name,
Memory address,0x0211b6336d30
Stoichiometry,nadh_c + succinyl_semialdehyde_c --> four_hydroxybutyrate_c + h_c + nad_c Nicotinamide adenine dinucleotide - reduced + --> + H+ + Nicotinamide adenine dinucleotide
GPR,
Lower bound,0.0
Upper bound,1000.0


In [11]:
hbd_coa_t = Reaction('HBCT') #4-hydroxybutyryl-CoA transferase
four_hbcoa = Metabolite(id='four_hbcoa_c', compartment='c')
hbd_coa_t.add_metabolites({model_akg.metabolites.accoa_c: -1,
                              model_akg.metabolites.ac_c: 1,
                              four_hbcoa: 1, #new one
                              four_hydroxybutyrate: -1
                             })
print(hbd_coa_t.build_reaction_string())
model_akg.add_reactions([hbd_coa_t])
model_akg.reactions.HBCT

accoa_c + four_hydroxybutyrate_c --> ac_c + four_hbcoa_c


Reaction identifier,HBCT
Name,
Memory address,0x0211b6336df0
Stoichiometry,accoa_c + four_hydroxybutyrate_c --> ac_c + four_hbcoa_c Acetyl-CoA + --> Acetate +
GPR,
Lower bound,0.0
Upper bound,1000.0


In [12]:
hbd_coa_r = Reaction('HBCR') #4-hydroxybutyryl-CoA reductase
four_hbaldehyde = Metabolite(id='four_hbaldehyde_c', compartment='c')
hbd_coa_r.add_metabolites({model_akg.metabolites.h_c: 1,
                              model_akg.metabolites.nadh_c: -1,
                              four_hbaldehyde: 1, #new one
                              four_hbcoa: -1,
                              model_akg.metabolites.nad_c: 1,
                              model_akg.metabolites.coa_c: 1
                             })
print(hbd_coa_r.build_reaction_string())
model_akg.add_reactions([hbd_coa_r])
model_akg.reactions.HBCR

four_hbcoa_c + nadh_c --> coa_c + four_hbaldehyde_c + h_c + nad_c


Reaction identifier,HBCR
Name,
Memory address,0x0211b632f880
Stoichiometry,four_hbcoa_c + nadh_c --> coa_c + four_hbaldehyde_c + h_c + nad_c + Nicotinamide adenine dinucleotide - reduced --> Coenzyme A + + H+ + Nicotinamide adenine dinucleotide
GPR,
Lower bound,0.0
Upper bound,1000.0


In [17]:
adh= Reaction('ADH') #alcohol dehydrogenase
bdo = Metabolite(id='bdo_c', compartment='c')
adh.add_metabolites({model_akg.metabolites.h_c: 1,
                              model_akg.metabolites.nadh_c: -1,
                              four_hbaldehyde: -1,
                              bdo: 1, #new one
                              model_akg.metabolites.nad_c: 1,
                             })
print(adh.build_reaction_string())
model_akg.add_reactions([adh])
model_akg.reactions.ADH

four_hbaldehyde_c + nadh_c --> bdo_c + h_c + nad_c


Reaction identifier,ADH
Name,
Memory address,0x0211b6cadfd0
Stoichiometry,four_hbaldehyde_c + nadh_c --> bdo_c + h_c + nad_c + Nicotinamide adenine dinucleotide - reduced --> + H+ + Nicotinamide adenine dinucleotide
GPR,
Lower bound,0.0
Upper bound,1000.0


In [21]:
bdo_exchange = Reaction('EX_bdo')
bdo_exchange.add_metabolites({model_akg.metabolites.bdo_c: -1})
model_akg.add_reaction(bdo_exchange)
model_akg.add_boundary(model_akg.metabolites.bdo_c, type='demand')

Ignoring reaction 'EX_bdo' since it already exists.


ValueError: Boundary reaction 'DM_bdo_c' already exists.

Same as with Sucd model, the Akg model is saved in a .json file and later vizualised using Escher.

In [19]:
cobra.io.save_json_model(model_akg, "akg.json")

### Yield Calulation 

Here the yield of Akg Model is calculated. Similarly, the units of the yield correspont to (produced/glucose)*mmol. And again at this point, it is an uptake reactions that is also why the calculation of the uptake is negative. 

In [24]:
with model:
    model_akg.objective=model_akg.reactions.ADH
    optimized = float(model_akg.optimize().objective_value)
    print(optimized)

9.708085106382972


In [25]:
uptake_akg = model_akg.reactions.EX_glc__D_e.flux #uptake <--
print("Uptake:", uptake_akg)

Uptake: -10.0


In [26]:
Models_yield['akg']= optimized
yield_akg = (-1)*(Models_yield['akg'])/uptake

In [27]:
print("Yield for model Akg =",yield_akg) # Yield for Model_akg (a-ketoglutarate) p-mmol/gl-mmol
#it is multiplied by -1 because the reaction is an uptake which means thats goes for the right to the left

Yield for model Akg = 0.9708085106382972
